In [1]:
#!nvidia-smi

In [2]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm

import torch
import torchvision

print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA is available:", torch.cuda.is_available())

PyTorch version: 2.0.0+cu118
Torchvision version: 0.15.1+cu118
CUDA is available: True


In [3]:
import sys
!{sys.executable} -m pip install opencv-python matplotlib
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

#!git clone https://github.com/facebookresearch/segment-anything.git
#!pip install git+https://github.com/facebookresearch/segment-anything.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-ugx07gs6
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-ugx07gs6
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36610 sha256=14698f6703ca67de79939d7f76d5c25734f2f36237bb8b13cb87e0c766f387ab
  Stored in directory: /tmp/pip-ephem-wheel-cache-_n0trydt/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment-anything


In [4]:
# %cd segment-anything
# !pip install -e .
# %cd ..

In [5]:
#!pip install opencv-python pycocotools matplotlib onnxruntime onnx

In [6]:
# Get TACO dataset

import os
from google.colab import drive
drive.mount('/gdrive',force_remount=True)
if not os.path.isfile('/content/TACO.zip'):
  %cp /gdrive/MyDrive/TACO.zip /content/TACO.zip
  # unzip data, move them 
  !unzip -qq ./TACO.zip 
  %mkdir -p ./dataset/images/ && mv ./TACO/* ./dataset/images/ && rm -rf ./TACO

Mounted at /gdrive


In [7]:
#!wget https://github.com/pedropro/TACO/raw/master/data/annotations.json

In [8]:
if not os.path.isfile('./sam_vit_h_4b8939.pth') and os.path.isfile('/gdrive/MyDrive/sam_vit_h_4b8939.pth'):
  !cp /gdrive/MyDrive/sam_vit_h_4b8939.pth ./
else:
  !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
  !cp sam_vit_h_4b8939.pth /gdrive/MyDrive/

In [9]:
import json
if not os.path.isdir('./SAM_output'):
  !mkdir SAM_output

In [10]:
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)
    polygons = []
    color = []
    for ann in sorted_anns:
        m = ann['segmentation']
        img = np.ones((m.shape[0], m.shape[1], 3))
        color_mask = np.random.random((1, 3)).tolist()[0]
        for i in range(3):
            img[:,:,i] = color_mask[i]
        ax.imshow(np.dstack((img, m*0.35)))

## Get SAM, load pretrained weights

In [11]:
#import sys
#sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)



In [18]:
def img_mask_getter(img_id):
  image = cv2.imread('/content/dataset/images/'+str(img_id)+'.jpg')
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  masks = mask_generator.generate(image)
  np.savez_compressed('/content/SAM_output/'+str(img_id)+'.npz',masks)
  

  

In [19]:
#get cropped objects
def objects_getter(image,masks):
  candidates = []
  for id, mask in enumerate(masks):
    x,y,w,h = mask['bbox']
    masked = image*np.dstack([mask['segmentation'],mask['segmentation'],mask['segmentation']])
    masked_cropped = masked[y:y+h,x:x+w]
    cv2.imwrite(str(id)+'.png',masked_cropped)
    candidates.append(masked_cropped)
  return candidates

## Make segmentations on images

In [22]:
# DEMO: get segmentation masks for the first 50 images;
# set range() to 1500 for results on all 1500 TACO images

!rm -rf /content/SAM_output/*
for i in tqdm(range(50)):
  img_mask_getter(i)
  

100%|██████████| 50/50 [05:18<00:00,  6.37s/it]
